In [1]:
import sys
sys.path.append('G:\我的雲端硬碟\python')
import numpy as np
from polib.CsvEngn import *
from MLB_functions import *
from tqdm import tqdm

import statsmodels.api as sm #OLS
# https://www.statsmodels.org/stable/generated/statsmodels.regression.linear_model.OLS.html

from linearmodels.panel import PanelOLS # FE
# https://bashtage.github.io/linearmodels/doc/panel/models.html

# 讀檔

In [2]:
#野手
df_bat = pd.read_excel(r"G:\我的雲端硬碟\python\output\0911\Spotrac_bat_data_20210911_13.56.21.xlsx",
                        engine='openpyxl')
# 投手
df_pitch= pd.read_excel(r"G:\我的雲端硬碟\python\output\0911\Spotrac_pitch_data_20210911_13.56.33.xlsx",
                      engine='openpyxl')

# 特徵轉換

In [3]:
# # is_2020+2021
df_bat.loc[:,"is_gt_2020"] = (df_bat.loc[:,"Season"]>=2020)
df_pitch.loc[:,"is_gt_2020"] = (df_pitch.loc[:,"Season"]>=2020)

In [4]:
# for df in [df_pitch, df_bat]:
for df in [df_pitch,df_bat]:
    
    #T,F transform
    for col in [col for col in df_bat.columns if "is_" in col]:
        df.loc[:,col] = df.loc[:,col].apply(lambda v: 1 if v else 0)
        
    # AAV_dollar 取 自然對數
    df.loc[:,"AAV_dollar"] = np.log(df.loc[:,"AAV_dollar"])
    df.loc[np.isinf(df_bat.loc[:,"AAV_dollar"]),"AAV_dollar"] = np.nan
    
#     NewCBA 2017~2021 dummy
    df.loc[:,"NewCBA"] =df.loc[:,"Season"].apply(lambda s: 1 if (int(s)>=2017) & (int(s)<=2021) else 0)
    

# 年度篩選

In [5]:
# 篩選
df_pitch = df_pitch[(df_pitch.loc[:,"Season"]<=2021) & (df_pitch.loc[:,"Season"]>=2012)]
print(len(df_pitch))
# df_pitch = df_pitch[(df_pitch.loc[:,"Season"]<=2016) & (df_pitch.loc[:,"Season"]>=2012)]
# df_pitch = df_pitch[(df_pitch.loc[:,"Season"]>=2017) & (df_pitch.loc[:,"Season"]<=2021)]
print(len(df_pitch))

2487
2487


# Model1

In [6]:
#     首先以合約剩餘年數作為主要的解釋變數，提出的普通最小平方法模型 (Ordinary Least Squares Method) 如下：

# WAR = β1 Contract Years Remaining + One Year Contract + Age +  Age2 + Position + ln (Average Salary) + Team + ChangedTeam + Last Year + 2020season + 2021season + ε 
# ……… (1)


##  投手

In [7]:
df_pic = df_pitch.copy(deep=True) # 投手
# df_pic = df_bat.copy(deep=True) # 野手
df_pic = df_pic[(df_pic.loc[:,"Season"]<=2021) & (df_pic.loc[:,"Season"]>=2012)]
print(df_pic.Season.unique())


X_cols = df_pic.columns.tolist()
remove_lst = [
 '_idx',
 'Season',
 'Player',
 'Team',
 'WAR',
 'WAR_lag_1_MA3',
 'WAR_SHIRK',
#  'is_changeTeam',
#  'Age',
#  'Age^2',
 'Pos',
 'YRS',
#  'YRS_left',
#  'AAV_dollar',
#  'is_1year_contract',
 'is_FIRST_year_BigContract',
#  'is_END_year',
 'is_left_2years',
 'is_left_3years',
 'is_left_ge4years',
 "type",
 "Player_ID",
 "NewCBA",
    
]

for col in remove_lst:
    X_cols.remove(col)

# dropna
print(len(df_pic))
df_pic = df_pic[X_cols+["WAR"]].replace([np.inf, -np.inf], np.nan).dropna()
print(len(df_pic))

X = df_pic[X_cols].copy(deep=True)
y = df_pic["WAR"].copy(deep=True)

# OLS
X = sm.add_constant(X)
est = sm.OLS(y, X.astype(float)).fit()
est.summary()

[2012 2013 2014 2015 2016 2017 2018 2019 2020 2021]
2487
921


C:\Users\EMCT_Jeff\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    WAR   R-squared:                       0.256
Model:                            OLS   Adj. R-squared:                  0.224
Method:                 Least Squares   F-statistic:                     7.993
Date:                Sat, 11 Sep 2021   Prob (F-statistic):           4.88e-36
Time:                        14:04:58   Log-Likelihood:                -1532.7
No. Observations:                 921   AIC:                             3143.
Df Residuals:                     882   BIC:                             3332.
Df Model:                          38                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 7.0301      4.286      1.640      0.101      -1.383      15.443
is_changeTeam        -0.3971      0.109     -3.636      0.000      -0.611      -0.183
Age                  -0.5237      0.251     -2.091      0.037      -1.015      -0.032
Age^2                 0.0080      0.004      2.164      0.031       0.001       0.015
YRS_left              0.2025      0.051      3.991      0.000       0.103       0.302
AAV_dollar            0.1323      0.073      1.808      0.071      -0.011       0.276
is_1year_contract     0.0373      0.130      0.288      0.774      -0.217       0.292
is_END_year          -0.9914      0.142     -6.966      0.000      -1.271      -0.712
Team_ATL              0.3121      0.362      0.863      0.388      -0.398       1.022
Team_BAL             -0.3466      0.467     -0.743      0.458      -1.263       0.569
Team_BOS              0.4990      0.355      1.404      0.161      -0.199       1.197
Team_CHC              0.0210      0.333      0.063      0.950      -0.632       0.674
Team_CHW              0.3890      0.359      1.083      0.279      -0.316       1.094
Team_CIN             -0.3379      0.364     -0.928      0.354      -1.053       0.377
Team_CLE              0.4735      0.391      1.211      0.226      -0.294       1.241
Team_COL             -0.4959      0.380     -1.307      0.192      -1.241       0.249
Team_DET             -0.3132      0.367     -0.853      0.394      -1.034       0.408
Team_HOU              0.3339      0.348      0.958      0.338      -0.350       1.018
Team_KC              -0.0282      0.348     -0.081      0.935      -0.711       0.654
Team_LAA             -0.3584      0.372     -0.963      0.336      -1.089       0.372
Team_LAD             -0.1487      0.338     -0.439      0.661      -0.813       0.516
Team_MIA             -0.2768      0.384     -0.722      0.471      -1.029       0.476
Team_MIL              0.0070      0.350      0.020      0.984      -0.681       0.695
Team_MIN             -0.1747      0.350     -0.499      0.618      -0.862       0.512
Team_NYM             -0.4123      0.352     -1.171      0.242      -1.103       0.279
Team_NYY              0.4776      0.366      1.304      0.192      -0.241       1.196
Team_OAK              0.7413      0.354      2.095      0.036       0.047       1.436
Team_PHI              0.2987      0.352      0.849      0.396      -0.392       0.989
Team_PIT              0.3095      0.445      0.696      0.487      -0.564       1.183
Team_SD              -0.1438      0.360     -0.399      0.690      -0.851       0.563
Team_SEA              0.2532      0.372      0.681      0.496      -0.476       0.983
Team_SF              -0.1300      0.343     -0.379      0.705      -0.803       0.543
Team_STL             -0.4345      0.373     -1.164      0.245      -1.167

## 野手

In [8]:
# df_pic = df_pitch.copy(deep=True) # 投手
df_pic = df_bat.copy(deep=True) # 野手
df_pic = df_pic[(df_pic.loc[:,"Season"]<=2021) & (df_pic.loc[:,"Season"]>=2012)]
print(df_pic.Season.unique())


X_cols = df_pic.columns.tolist()
remove_lst = [
 '_idx',
 'Season',
 'Player',
 'Team',
 'WAR',
 'WAR_lag_1_MA3',
 'WAR_SHIRK',
#  'is_changeTeam',
#  'Age',
#  'Age^2',
 'Pos',
 'YRS',
#  'YRS_left',
#  'AAV_dollar',
#  'is_1year_contract',
 'is_FIRST_year_BigContract',
#  'is_END_year',
 'is_left_2years',
 'is_left_3years',
 'is_left_ge4years',
 "type",
 "Player_ID",
 "NewCBA",
    
]

for col in remove_lst:
    X_cols.remove(col)

# dropna
print(len(df_pic))
df_pic = df_pic[X_cols+["WAR"]].replace([np.inf, -np.inf], np.nan).dropna()
print(len(df_pic))

X = df_pic[X_cols].copy(deep=True)
y = df_pic["WAR"].copy(deep=True)

# OLS
X = sm.add_constant(X)
est = sm.OLS(y, X.astype(float)).fit()
est.summary()

[2012 2013 2014 2015 2016 2017 2018 2019 2020 2021]
2496
929


C:\Users\EMCT_Jeff\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    WAR   R-squared:                       0.335
Model:                            OLS   Adj. R-squared:                  0.300
Method:                 Least Squares   F-statistic:                     9.656
Date:                Sat, 11 Sep 2021   Prob (F-statistic):           7.05e-52
Time:                        14:04:58   Log-Likelihood:                -1671.3
No. Observations:                 929   AIC:                             3437.
Df Residuals:                     882   BIC:                             3664.
Df Model:                          46                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                11.3767      3.696      3.078      0.002       4.122      18.631
is_changeTeam        -0.5636      0.130     -4.350      0.000      -0.818      -0.309
Age                  -0.9198      0.221     -4.170      0.000      -1.353      -0.487
Age^2                 0.0123      0.003      3.699      0.000       0.006       0.019
YRS_left              0.0705      0.037      1.901      0.058      -0.002       0.143
AAV_dollar            0.4696      0.076      6.203      0.000       0.321       0.618
is_1year_contract     0.2162      0.153      1.409      0.159      -0.085       0.517
is_END_year          -1.2009      0.158     -7.581      0.000      -1.512      -0.890
Team_ATL             -0.6632      0.346     -1.919      0.055      -1.342       0.015
Team_BAL             -1.1115      0.421     -2.642      0.008      -1.937      -0.286
Team_BOS             -0.3391      0.340     -0.998      0.319      -1.006       0.328
Team_CHC             -0.5555      0.376     -1.478      0.140      -1.293       0.182
Team_CHW             -0.6106      0.393     -1.554      0.121      -1.382       0.161
Team_CIN             -0.3277      0.378     -0.867      0.386      -1.070       0.414
Team_CLE             -0.5417      0.351     -1.542      0.123      -1.231       0.148
Team_COL             -1.3586      0.393     -3.453      0.001      -2.131      -0.586
Team_DET             -0.8714      0.393     -2.219      0.027      -1.642      -0.101
Team_HOU              0.2987      0.388      0.770      0.442      -0.463       1.060
Team_KC              -0.4561      0.399     -1.142      0.254      -1.240       0.328
Team_LAA             -0.4680      0.350     -1.336      0.182      -1.156       0.220
Team_LAD              0.6099      0.389      1.566      0.118      -0.154       1.374
Team_MIA             -0.8540      0.388     -2.202      0.028      -1.615      -0.093
Team_MIL              0.0702      0.363      0.193      0.847      -0.643       0.783
Team_MIN             -0.5601      0.421     -1.330      0.184      -1.386       0.266
Team_NYM             -0.2670      0.357     -0.747      0.455      -0.969       0.435
Team_NYY             -0.1906      0.353     -0.540      0.589      -0.883       0.502
Team_OAK             -0.0769      0.415     -0.185      0.853      -0.891       0.737
Team_PHI             -0.3630      0.400     -0.907      0.364      -1.148       0.422
Team_PIT             -0.1229      0.426     -0.288      0.773      -0.959       0.713
Team_SD              -0.9334      0.415     -2.250      0.025      -1.748      -0.119
Team_SEA              0.4200      0.403      1.043      0.297      -0.371       1.211
Team_SF              -0.8686      0.381     -2.280      0.023      -1.616      -0.121
Team_STL             -0.2127      0.365     -0.583      0.560      -0.928

### 前5年

In [9]:
# df_pic = df_pitch.copy(deep=True) # 投手
df_pic = df_bat.copy(deep=True) # 野手
df_pic = df_pic[(df_pic.loc[:,"Season"]<=2016) & (df_pic.loc[:,"Season"]>=2012)]
print(df_pic.Season.unique())


X_cols = df_pic.columns.tolist()
remove_lst = [
 '_idx',
 'Season',
 'Player',
 'Team',
 'WAR',
 'WAR_lag_1_MA3',
 'WAR_SHIRK',
#  'is_changeTeam',
#  'Age',
#  'Age^2',
 'Pos',
 'YRS',
#  'YRS_left',
#  'AAV_dollar',
#  'is_1year_contract',
 'is_FIRST_year_BigContract',
#  'is_END_year',
 'is_left_2years',
 'is_left_3years',
 'is_left_ge4years',
 "type",
 "Player_ID",
 "NewCBA",
    
]

for col in remove_lst:
    X_cols.remove(col)

# dropna
print(len(df_pic))
df_pic = df_pic[X_cols+["WAR"]].replace([np.inf, -np.inf], np.nan).dropna()
print(len(df_pic))

X = df_pic[X_cols].copy(deep=True)
y = df_pic["WAR"].copy(deep=True)

# OLS
X = sm.add_constant(X)
est = sm.OLS(y, X.astype(float)).fit()
est.summary()

[2012 2013 2014 2015 2016]
1599
421


C:\Users\EMCT_Jeff\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    WAR   R-squared:                       0.376
Model:                            OLS   Adj. R-squared:                  0.301
Method:                 Least Squares   F-statistic:                     5.028
Date:                Sat, 11 Sep 2021   Prob (F-statistic):           3.57e-19
Time:                        14:04:59   Log-Likelihood:                -759.86
No. Observations:                 421   AIC:                             1612.
Df Residuals:                     375   BIC:                             1798.
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 6.5370      5.453      1.199      0.231      -4.186      17.260
is_changeTeam        -0.5539      0.204     -2.711      0.007      -0.956      -0.152
Age                  -0.6989      0.327     -2.138      0.033      -1.341      -0.056
Age^2                 0.0100      0.005      2.024      0.044       0.000       0.020
YRS_left              0.1907      0.065      2.915      0.004       0.062       0.319
AAV_dollar            0.4842      0.119      4.062      0.000       0.250       0.719
is_1year_contract     0.3481      0.246      1.417      0.157      -0.135       0.831
is_END_year          -1.2408      0.249     -4.974      0.000      -1.731      -0.750
Team_ATL             -1.4907      0.586     -2.544      0.011      -2.643      -0.339
Team_BAL             -0.3027      0.729     -0.415      0.678      -1.735       1.130
Team_BOS             -0.3032      0.549     -0.552      0.581      -1.383       0.776
Team_CHC             -0.8028      0.601     -1.336      0.182      -1.984       0.378
Team_CHW             -0.8557      0.607     -1.410      0.159      -2.049       0.337
Team_CIN             -0.8897      0.572     -1.554      0.121      -2.015       0.236
Team_CLE             -1.0470      0.583     -1.796      0.073      -2.193       0.099
Team_COL             -1.1457      0.650     -1.763      0.079      -2.424       0.132
Team_DET             -0.8132      0.612     -1.328      0.185      -2.017       0.391
Team_HOU             -0.1231      0.753     -0.163      0.870      -1.604       1.358
Team_KC              -1.0416      0.698     -1.493      0.136      -2.413       0.330
Team_LAA             -0.7737      0.604     -1.281      0.201      -1.962       0.414
Team_LAD              0.3843      0.653      0.588      0.557      -0.900       1.668
Team_MIA             -1.2345      0.592     -2.086      0.038      -2.398      -0.071
Team_MIL             -0.3738      0.587     -0.637      0.525      -1.528       0.780
Team_MIN             -0.9336      0.676     -1.380      0.168      -2.264       0.397
Team_NYM              0.2373      0.611      0.388      0.698      -0.965       1.440
Team_NYY             -1.0829      0.549     -1.971      0.049      -2.163      -0.003
Team_OAK             -0.4076      0.671     -0.607      0.544      -1.727       0.912
Team_PHI             -0.5689      0.751     -0.758      0.449      -2.046       0.908
Team_PIT             -0.0461      0.635     -0.073      0.942      -1.295       1.203
Team_SD              -1.0040      0.686     -1.463      0.144      -2.354       0.346
Team_SEA             -0.1827      0.604     -0.303      0.762      -1.371       1.005
Team_SF              -1.1452      0.664     -1.724      0.086      -2.451       0.161
Team_STL             -0.0921      0.582     -0.158      0.874      -1.236

### 後5年

In [10]:
# df_pic = df_pitch.copy(deep=True) # 投手
df_pic = df_bat.copy(deep=True) # 野手
df_pic = df_pic[(df_pic.loc[:,"Season"]<=2021) & (df_pic.loc[:,"Season"]>=2017)]
print(df_pic.Season.unique())


X_cols = df_pic.columns.tolist()
remove_lst = [
 '_idx',
 'Season',
 'Player',
 'Team',
 'WAR',
 'WAR_lag_1_MA3',
 'WAR_SHIRK',
#  'is_changeTeam',
#  'Age',
#  'Age^2',
 'Pos',
 'YRS',
#  'YRS_left',
#  'AAV_dollar',
#  'is_1year_contract',
 'is_FIRST_year_BigContract',
#  'is_END_year',
 'is_left_2years',
 'is_left_3years',
 'is_left_ge4years',
 "type",
 "Player_ID",
 "NewCBA",
    
]

for col in remove_lst:
    X_cols.remove(col)

# dropna
print(len(df_pic))
df_pic = df_pic[X_cols+["WAR"]].replace([np.inf, -np.inf], np.nan).dropna()
print(len(df_pic))

X = df_pic[X_cols].copy(deep=True)
y = df_pic["WAR"].copy(deep=True)

# OLS
X = sm.add_constant(X)
est = sm.OLS(y, X.astype(float)).fit()
est.summary()

[2017 2018 2019 2020 2021]
897
508


C:\Users\EMCT_Jeff\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    WAR   R-squared:                       0.384
Model:                            OLS   Adj. R-squared:                  0.322
Method:                 Least Squares   F-statistic:                     6.238
Date:                Sat, 11 Sep 2021   Prob (F-statistic):           7.32e-27
Time:                        14:04:59   Log-Likelihood:                -877.19
No. Observations:                 508   AIC:                             1848.
Df Residuals:                     461   BIC:                             2047.
Df Model:                          46                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                18.5847      5.289      3.514      0.000       8.192      28.978
is_changeTeam        -0.5076      0.171     -2.963      0.003      -0.844      -0.171
Age                  -1.2700      0.314     -4.047      0.000      -1.887      -0.653
Age^2                 0.0167      0.005      3.534      0.000       0.007       0.026
YRS_left              0.0016      0.047      0.035      0.972      -0.090       0.093
AAV_dollar            0.4379      0.107      4.077      0.000       0.227       0.649
is_1year_contract     0.0848      0.201      0.422      0.674      -0.310       0.480
is_END_year          -1.0728      0.210     -5.117      0.000      -1.485      -0.661
Team_ATL             -0.1269      0.426     -0.298      0.766      -0.965       0.711
Team_BAL             -1.5282      0.517     -2.954      0.003      -2.545      -0.512
Team_BOS             -0.6029      0.435     -1.385      0.167      -1.458       0.253
Team_CHC             -0.2653      0.490     -0.542      0.588      -1.228       0.697
Team_CHW             -0.3827      0.544     -0.704      0.482      -1.451       0.686
Team_CIN              0.2289      0.547      0.418      0.676      -0.847       1.304
Team_CLE             -0.2699      0.448     -0.602      0.547      -1.151       0.611
Team_COL             -1.5064      0.494     -3.050      0.002      -2.477      -0.536
Team_DET             -1.0812      0.532     -2.030      0.043      -2.128      -0.035
Team_HOU              0.6630      0.454      1.462      0.144      -0.228       1.554
Team_KC              -0.2151      0.486     -0.443      0.658      -1.170       0.740
Team_LAA             -0.3688      0.436     -0.845      0.398      -1.226       0.489
Team_LAD              0.8758      0.487      1.800      0.073      -0.081       1.832
Team_MIA             -0.6079      0.534     -1.139      0.255      -1.657       0.441
Team_MIL              0.3575      0.462      0.775      0.439      -0.549       1.264
Team_MIN             -0.3041      0.551     -0.552      0.581      -1.386       0.778
Team_NYM             -0.5788      0.441     -1.312      0.190      -1.445       0.288
Team_NYY              0.5840      0.477      1.224      0.222      -0.354       1.522
Team_OAK              0.1237      0.541      0.229      0.819      -0.940       1.187
Team_PHI             -0.1672      0.476     -0.351      0.726      -1.103       0.769
Team_PIT             -0.3070      0.603     -0.509      0.611      -1.491       0.877
Team_SD              -0.8980      0.538     -1.669      0.096      -1.956       0.160
Team_SEA              0.7691      0.593      1.297      0.195      -0.396       1.934
Team_SF              -0.7176      0.467     -1.537      0.125      -1.635       0.200
Team_STL             -0.2514      0.471     -0.534      0.594      -1.177

# Model 2

In [11]:
#     為了控制上述的問題，在模型 (2) 改以固定效果模型 (Fixed Effect Model)取代原先的 OLS 模型：


# WARit = β1 Contract Years Remainingit +β2 One Year Contractit + Ageit + Ageit2 + Positionit + ln (Average Salary)it + Teamit + ChangedTeamit + Last Yearit +  2020seasonit +  2021seasonit + αi + λt + εit 
# ……… (2)


## 投手

In [12]:
df_pic = df_pitch.copy(deep=True) # 投手
# df_pic = df_bat.copy(deep=True) # 野手
df_pic = df_pic[(df_pic.loc[:,"Season"]<=2021) & (df_pic.loc[:,"Season"]>=2012)]
print(df_pic.Season.unique())


X_cols = df_pic.columns.tolist()
remove_lst = [
 '_idx',
#  'Season',
 'Player',
 'Team',
 'WAR',
 'WAR_lag_1_MA3',
 'WAR_SHIRK',
#  'is_changeTeam',
#  'Age',
#  'Age^2',
 'Pos',
 'YRS',
#  'YRS_left',
#  'AAV_dollar',
#  'is_1year_contract',
 'is_FIRST_year_BigContract',
#  'is_END_year',
 'is_left_2years',
 'is_left_3years',
 'is_left_ge4years',
 "type",
#  "Player_ID",
     "NewCBA"
]

ID_cols = ["Player_ID", "Season"]

for col in remove_lst:
    X_cols.remove(col)

# dropna
print(len(df_pic))
df_pic = df_pic[X_cols+["WAR"]].replace([np.inf, -np.inf], np.nan).dropna()
print(len(df_pic))

X = df_pic[X_cols].copy(deep=True)
y = df_pic[["WAR"]+ID_cols].copy(deep=True)

# FE
X = X.set_index(ID_cols)
X = sm.add_constant(X)

y = y.set_index(ID_cols)

fe = PanelOLS(y, X.astype(float), 
               entity_effects=True, 
               drop_absorbed=True,
               check_rank =False
              )
res = fe.fit(cov_type='clustered', cluster_entity=True)
print(res)

[2012 2013 2014 2015 2016 2017 2018 2019 2020 2021]
2487


C:\Users\EMCT_Jeff\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


921


C:\Users\EMCT_Jeff\anaconda3\lib\site-packages\linearmodels\panel\model.py:1796: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

Pos_SP

  warnings.warn(


                          PanelOLS Estimation Summary                           
Dep. Variable:                    WAR   R-squared:                        0.1799
Estimator:                   PanelOLS   R-squared (Between):             -0.3523
No. Observations:                 921   R-squared (Within):               0.1799
Date:                Sat, Sep 11 2021   R-squared (Overall):             -0.0430
Time:                        14:04:59   Log-likelihood                   -1283.4
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      3.1667
Entities:                         350   P-value                           0.0000
Avg Obs:                       2.6314   Distribution:                  F(37,534)
Min Obs:                       1.0000                                           
Max Obs:                       9.0000   F-statistic (robust):             5.4013
                            

## 野手

In [13]:
# df_pic = df_pitch.copy(deep=True) # 投手
df_pic = df_bat.copy(deep=True) # 野手
df_pic = df_pic[(df_pic.loc[:,"Season"]<=2021) & (df_pic.loc[:,"Season"]>=2012)]
print(df_pic.Season.unique())


X_cols = df_pic.columns.tolist()
remove_lst = [
 '_idx',
#  'Season',
 'Player',
 'Team',
 'WAR',
 'WAR_lag_1_MA3',
 'WAR_SHIRK',
#  'is_changeTeam',
#  'Age',
#  'Age^2',
 'Pos',
 'YRS',
#  'YRS_left',
#  'AAV_dollar',
#  'is_1year_contract',
 'is_FIRST_year_BigContract',
#  'is_END_year',
 'is_left_2years',
 'is_left_3years',
 'is_left_ge4years',
 "type",
#  "Player_ID",
     "NewCBA"
]

ID_cols = ["Player_ID", "Season"]

for col in remove_lst:
    X_cols.remove(col)

# dropna
print(len(df_pic))
df_pic = df_pic[X_cols+["WAR"]].replace([np.inf, -np.inf], np.nan).dropna()
print(len(df_pic))

X = df_pic[X_cols].copy(deep=True)
y = df_pic[["WAR"]+ID_cols].copy(deep=True)

# FE
X = X.set_index(ID_cols)
X = sm.add_constant(X)

y = y.set_index(ID_cols)

fe = PanelOLS(y, X.astype(float), 
               entity_effects=True, 
               drop_absorbed=True,
               check_rank =False
              )
res = fe.fit(cov_type='clustered', cluster_entity=True)
print(res)

[2012 2013 2014 2015 2016 2017 2018 2019 2020 2021]
2496
929
                          PanelOLS Estimation Summary                           
Dep. Variable:                    WAR   R-squared:                        0.3176
Estimator:                   PanelOLS   R-squared (Between):             -1.6960
No. Observations:                 929   R-squared (Within):               0.3176
Date:                Sat, Sep 11 2021   R-squared (Overall):             -0.3207
Time:                        14:04:59   Log-likelihood                   -1360.8
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      6.9825
Entities:                         337   P-value                           0.0000
Avg Obs:                       2.7567   Distribution:                  F(37,555)
Min Obs:                       1.0000                                           
Max Obs:                      10.0000   F-statis

C:\Users\EMCT_Jeff\anaconda3\lib\site-packages\linearmodels\panel\model.py:1796: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

Pos_2B, Pos_3B, Pos_C, Pos_CF, Pos_DH, Pos_LF, Pos_OF, Pos_RF, Pos_SS

  warnings.warn(


### 前5年

In [14]:
# df_pic = df_pitch.copy(deep=True) # 投手
df_pic = df_bat.copy(deep=True) # 野手
df_pic = df_pic[(df_pic.loc[:,"Season"]<=2016) & (df_pic.loc[:,"Season"]>=2012)]
print(df_pic.Season.unique())


X_cols = df_pic.columns.tolist()
remove_lst = [
 '_idx',
#  'Season',
 'Player',
 'Team',
 'WAR',
 'WAR_lag_1_MA3',
 'WAR_SHIRK',
#  'is_changeTeam',
#  'Age',
#  'Age^2',
 'Pos',
 'YRS',
#  'YRS_left',
#  'AAV_dollar',
#  'is_1year_contract',
 'is_FIRST_year_BigContract',
#  'is_END_year',
 'is_left_2years',
 'is_left_3years',
 'is_left_ge4years',
 "type",
#  "Player_ID",
     "NewCBA"
]

ID_cols = ["Player_ID", "Season"]

for col in remove_lst:
    X_cols.remove(col)

# dropna
print(len(df_pic))
df_pic = df_pic[X_cols+["WAR"]].replace([np.inf, -np.inf], np.nan).dropna()
print(len(df_pic))

X = df_pic[X_cols].copy(deep=True)
y = df_pic[["WAR"]+ID_cols].copy(deep=True)

# FE
X = X.set_index(ID_cols)
X = sm.add_constant(X)

y = y.set_index(ID_cols)

fe = PanelOLS(y, X.astype(float), 
               entity_effects=True, 
               drop_absorbed=True,
               check_rank =False
              )
res = fe.fit(cov_type='clustered', cluster_entity=True)
print(res)

[2012 2013 2014 2015 2016]
1599
421
                          PanelOLS Estimation Summary                           
Dep. Variable:                    WAR   R-squared:                        0.2831
Estimator:                   PanelOLS   R-squared (Between):             -1.9750
No. Observations:                 421   R-squared (Within):               0.2831
Date:                Sat, Sep 11 2021   R-squared (Overall):             -1.2723
Time:                        14:05:00   Log-likelihood                   -565.51
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      2.0406
Entities:                         199   P-value                           0.0012
Avg Obs:                       2.1156   Distribution:                  F(36,186)
Min Obs:                       1.0000                                           
Max Obs:                       5.0000   F-statistic (robust):          5.

C:\Users\EMCT_Jeff\anaconda3\lib\site-packages\linearmodels\panel\model.py:1796: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

Pos_2B, Pos_3B, Pos_C, Pos_CF, Pos_DH, Pos_LF, Pos_OF, Pos_RF, Pos_SS, is_gt_2020

  warnings.warn(


### 後5年

In [15]:
# df_pic = df_pitch.copy(deep=True) # 投手
df_pic = df_bat.copy(deep=True) # 野手
df_pic = df_pic[(df_pic.loc[:,"Season"]<=2021) & (df_pic.loc[:,"Season"]>=2017)]
print(df_pic.Season.unique())


X_cols = df_pic.columns.tolist()
remove_lst = [
 '_idx',
#  'Season',
 'Player',
 'Team',
 'WAR',
 'WAR_lag_1_MA3',
 'WAR_SHIRK',
#  'is_changeTeam',
#  'Age',
#  'Age^2',
 'Pos',
 'YRS',
#  'YRS_left',
#  'AAV_dollar',
#  'is_1year_contract',
 'is_FIRST_year_BigContract',
#  'is_END_year',
 'is_left_2years',
 'is_left_3years',
 'is_left_ge4years',
 "type",
#  "Player_ID",
     "NewCBA"
]

ID_cols = ["Player_ID", "Season"]

for col in remove_lst:
    X_cols.remove(col)

# dropna
print(len(df_pic))
df_pic = df_pic[X_cols+["WAR"]].replace([np.inf, -np.inf], np.nan).dropna()
print(len(df_pic))

X = df_pic[X_cols].copy(deep=True)
y = df_pic[["WAR"]+ID_cols].copy(deep=True)

# FE
X = X.set_index(ID_cols)
X = sm.add_constant(X)

y = y.set_index(ID_cols)

fe = PanelOLS(y, X.astype(float), 
               entity_effects=True, 
               drop_absorbed=True,
               check_rank =False
              )
res = fe.fit(cov_type='clustered', cluster_entity=True)
print(res)

[2017 2018 2019 2020 2021]
897
508
                          PanelOLS Estimation Summary                           
Dep. Variable:                    WAR   R-squared:                        0.3691
Estimator:                   PanelOLS   R-squared (Between):             -1.2850
No. Observations:                 508   R-squared (Within):               0.3691
Date:                Sat, Sep 11 2021   R-squared (Overall):             -0.2201
Time:                        14:05:00   Log-likelihood                   -677.73
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      3.9685
Entities:                         220   P-value                           0.0000
Avg Obs:                       2.3091   Distribution:                  F(37,251)
Min Obs:                       1.0000                                           
Max Obs:                       5.0000   F-statistic (robust):             

# Model 3

In [16]:
# 模型 (3) 將主要的解釋變數由線性的合約剩餘年數替換為合約剩餘兩年、合約剩餘三年、合約剩餘四年以上的虛擬變數：

# WARit = Two Years Remainingit + Three Years Remainingit + FourPlus Years Remainingit + One Year Contractit + Ageit + Ageit2 + Positionit + ln (Average Salary)it + Teamit + ChangedTeamit + Last Yearit +  2020seasonit +  2021seasonit + αi + λt + εit 
# ……… (3)


## 投手

In [17]:
df_pic = df_pitch.copy(deep=True) # 投手
# df_pic = df_bat.copy(deep=True) # 野手
df_pic = df_pic[(df_pic.loc[:,"Season"]<=2021) & (df_pic.loc[:,"Season"]>=2012)]
print(df_pic.Season.unique())


X_cols = df_pic.columns.tolist()
remove_lst = [
 '_idx',
#  'Season',
 'Player',
 'Team',
 'WAR',
 'WAR_lag_1_MA3',
 'WAR_SHIRK',
#  'is_changeTeam',
#  'Age',
#  'Age^2',
 'Pos',
 'YRS',
 'YRS_left',
#  'AAV_dollar',
#  'is_1year_contract',
 'is_FIRST_year_BigContract',
#  'is_END_year',
#  'is_left_2years',
#  'is_left_3years',
#  'is_left_ge4years',
 "type",
#  "Player_ID",
     "NewCBA"
]

ID_cols = ["Player_ID", 
           "Season"
          ]

for col in remove_lst:
    X_cols.remove(col)

# dropna
print(len(df_pic))
df_pic = df_pic[X_cols+["WAR"]].replace([np.inf, -np.inf], np.nan).dropna()
print(len(df_pic))

X = df_pic[X_cols].copy(deep=True)
y = df_pic[["WAR"]+ID_cols].copy(deep=True)

# FE
X = X.set_index(ID_cols)
X = sm.add_constant(X)

y = y.set_index(ID_cols)

fe = PanelOLS(y, X.astype(float), 
               entity_effects=True, 
               drop_absorbed=True,
               check_rank =False
              )
res = fe.fit(cov_type='clustered', cluster_entity=True)
print(res)

[2012 2013 2014 2015 2016 2017 2018 2019 2020 2021]
2487
921
                          PanelOLS Estimation Summary                           
Dep. Variable:                    WAR   R-squared:                        0.1842
Estimator:                   PanelOLS   R-squared (Between):             -0.3106
No. Observations:                 921   R-squared (Within):               0.1842
Date:                Sat, Sep 11 2021   R-squared (Overall):             -0.0202
Time:                        14:05:01   Log-likelihood                   -1281.0
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      3.0803
Entities:                         350   P-value                           0.0000
Avg Obs:                       2.6314   Distribution:                  F(39,532)
Min Obs:                       1.0000                                           
Max Obs:                       9.0000   F-statis

## 野手

In [18]:
# df_pic = df_pitch.copy(deep=True) # 投手
df_pic = df_bat.copy(deep=True) # 野手
df_pic = df_pic[(df_pic.loc[:,"Season"]<=2021) & (df_pic.loc[:,"Season"]>=2012)]
print(df_pic.Season.unique())


X_cols = df_pic.columns.tolist()
remove_lst = [
 '_idx',
#  'Season',
 'Player',
 'Team',
 'WAR',
 'WAR_lag_1_MA3',
 'WAR_SHIRK',
#  'is_changeTeam',
#  'Age',
#  'Age^2',
 'Pos',
 'YRS',
 'YRS_left',
#  'AAV_dollar',
#  'is_1year_contract',
 'is_FIRST_year_BigContract',
#  'is_END_year',
#  'is_left_2years',
#  'is_left_3years',
#  'is_left_ge4years',
 "type",
#  "Player_ID",
     "NewCBA"
]

ID_cols = ["Player_ID", 
           "Season"
          ]

for col in remove_lst:
    X_cols.remove(col)

# dropna
print(len(df_pic))
df_pic = df_pic[X_cols+["WAR"]].replace([np.inf, -np.inf], np.nan).dropna()
print(len(df_pic))

X = df_pic[X_cols].copy(deep=True)
y = df_pic[["WAR"]+ID_cols].copy(deep=True)

# FE
X = X.set_index(ID_cols)
X = sm.add_constant(X)

y = y.set_index(ID_cols)

fe = PanelOLS(y, X.astype(float), 
               entity_effects=True, 
               drop_absorbed=True,
               check_rank =False
              )
res = fe.fit(cov_type='clustered', cluster_entity=True)
print(res)

[2012 2013 2014 2015 2016 2017 2018 2019 2020 2021]
2496
929
                          PanelOLS Estimation Summary                           
Dep. Variable:                    WAR   R-squared:                        0.3242
Estimator:                   PanelOLS   R-squared (Between):             -1.7120
No. Observations:                 929   R-squared (Within):               0.3242
Date:                Sat, Sep 11 2021   R-squared (Overall):             -0.3199
Time:                        14:05:01   Log-likelihood                   -1356.3
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      6.8029
Entities:                         337   P-value                           0.0000
Avg Obs:                       2.7567   Distribution:                  F(39,553)
Min Obs:                       1.0000                                           
Max Obs:                      10.0000   F-statis

### 前5年

In [19]:
# df_pic = df_pitch.copy(deep=True) # 投手
df_pic = df_bat.copy(deep=True) # 野手
df_pic = df_pic[(df_pic.loc[:,"Season"]<=2016) & (df_pic.loc[:,"Season"]>=2012)]
print(df_pic.Season.unique())


X_cols = df_pic.columns.tolist()
remove_lst = [
 '_idx',
#  'Season',
 'Player',
 'Team',
 'WAR',
 'WAR_lag_1_MA3',
 'WAR_SHIRK',
#  'is_changeTeam',
#  'Age',
#  'Age^2',
 'Pos',
 'YRS',
 'YRS_left',
#  'AAV_dollar',
#  'is_1year_contract',
 'is_FIRST_year_BigContract',
#  'is_END_year',
#  'is_left_2years',
#  'is_left_3years',
#  'is_left_ge4years',
 "type",
#  "Player_ID",
     "NewCBA"
]

ID_cols = ["Player_ID", 
           "Season"
          ]

for col in remove_lst:
    X_cols.remove(col)

# dropna
print(len(df_pic))
df_pic = df_pic[X_cols+["WAR"]].replace([np.inf, -np.inf], np.nan).dropna()
print(len(df_pic))

X = df_pic[X_cols].copy(deep=True)
y = df_pic[["WAR"]+ID_cols].copy(deep=True)

# FE
X = X.set_index(ID_cols)
X = sm.add_constant(X)

y = y.set_index(ID_cols)

fe = PanelOLS(y, X.astype(float), 
               entity_effects=True, 
               drop_absorbed=True,
               check_rank =False
              )
res = fe.fit(cov_type='clustered', cluster_entity=True)
print(res)

[2012 2013 2014 2015 2016]
1599
421
                          PanelOLS Estimation Summary                           
Dep. Variable:                    WAR   R-squared:                        0.2970
Estimator:                   PanelOLS   R-squared (Between):             -1.7785
No. Observations:                 421   R-squared (Within):               0.2970
Date:                Sat, Sep 11 2021   R-squared (Overall):             -1.1301
Time:                        14:05:01   Log-likelihood                   -561.40
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      2.0455
Entities:                         199   P-value                           0.0010
Avg Obs:                       2.1156   Distribution:                  F(38,184)
Min Obs:                       1.0000                                           
Max Obs:                       5.0000   F-statistic (robust):         -4.

### 後5年

In [20]:
# df_pic = df_pitch.copy(deep=True) # 投手
df_pic = df_bat.copy(deep=True) # 野手
df_pic = df_pic[(df_pic.loc[:,"Season"]<=2021) & (df_pic.loc[:,"Season"]>=2017)]
print(df_pic.Season.unique())


X_cols = df_pic.columns.tolist()
remove_lst = [
 '_idx',
#  'Season',
 'Player',
 'Team',
 'WAR',
 'WAR_lag_1_MA3',
 'WAR_SHIRK',
#  'is_changeTeam',
#  'Age',
#  'Age^2',
 'Pos',
 'YRS',
 'YRS_left',
#  'AAV_dollar',
#  'is_1year_contract',
 'is_FIRST_year_BigContract',
#  'is_END_year',
#  'is_left_2years',
#  'is_left_3years',
#  'is_left_ge4years',
 "type",
#  "Player_ID",
     "NewCBA"
]

ID_cols = ["Player_ID", 
           "Season"
          ]

for col in remove_lst:
    X_cols.remove(col)

# dropna
print(len(df_pic))
df_pic = df_pic[X_cols+["WAR"]].replace([np.inf, -np.inf], np.nan).dropna()
print(len(df_pic))

X = df_pic[X_cols].copy(deep=True)
y = df_pic[["WAR"]+ID_cols].copy(deep=True)

# FE
X = X.set_index(ID_cols)
X = sm.add_constant(X)

y = y.set_index(ID_cols)

fe = PanelOLS(y, X.astype(float), 
               entity_effects=True, 
               drop_absorbed=True,
               check_rank =False
              )
res = fe.fit(cov_type='clustered', cluster_entity=True)
print(res)

[2017 2018 2019 2020 2021]
897
508
                          PanelOLS Estimation Summary                           
Dep. Variable:                    WAR   R-squared:                        0.3625
Estimator:                   PanelOLS   R-squared (Between):             -1.2534
No. Observations:                 508   R-squared (Within):               0.3625
Date:                Sat, Sep 11 2021   R-squared (Overall):             -0.2103
Time:                        14:05:02   Log-likelihood                   -680.37
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      3.6307
Entities:                         220   P-value                           0.0000
Avg Obs:                       2.3091   Distribution:                  F(39,249)
Min Obs:                       1.0000                                           
Max Obs:                       5.0000   F-statistic (robust):             

# Model 4

In [21]:
# SHIRK = β1 First Year + One Year Contract + Age + Age2 + Position + ln (Average Salary) + Team + ChangedTeam + Last Year + 2020season + 2021season + ε……… (4)

## 投手

In [22]:
df_pic = df_pitch.copy(deep=True) # 投手
# df_pic = df_bat.copy(deep=True) # 野手
df_pic = df_pic[(df_pic.loc[:,"Season"]<=2021) & (df_pic.loc[:,"Season"]>=2012)]
print(df_pic.Season.unique())


X_cols = df_pic.columns.tolist()
remove_lst = [
 '_idx',
 'Season',
 'Player',
 'Team',
 'WAR',
 'WAR_lag_1_MA3',
 'WAR_SHIRK',
#  'is_changeTeam',
#  'Age',
#  'Age^2',
 'Pos',
 'YRS',
 'YRS_left',
#  'AAV_dollar',
#  'is_1year_contract',
#  'is_FIRST_year_BigContract',
#  'is_END_year',
 'is_left_2years',
 'is_left_3years',
 'is_left_ge4years',
 "type",
 "Player_ID",
     "NewCBA",
#     "is_gt_2020"
]

for col in remove_lst:
    X_cols.remove(col)

# dropna
print(len(df_pic))
df_pic = df_pic[X_cols+["WAR"]].replace([np.inf, -np.inf], np.nan).dropna()
print(len(df_pic))

X = df_pic[X_cols].copy(deep=True)
y = df_pic["WAR"].copy(deep=True)

# OLS
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

[2012 2013 2014 2015 2016 2017 2018 2019 2020 2021]
2487
921


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    WAR   R-squared:                       0.243
Model:                            OLS   Adj. R-squared:                  0.210
Method:                 Least Squares   F-statistic:                     7.443
Date:                Sat, 11 Sep 2021   Prob (F-statistic):           4.79e-33
Time:                        14:05:02   Log-Likelihood:                -1540.9
No. Observations:                 921   AIC:                             3160.
Df Residuals:                     882   BIC:                             3348.
Df Model:                          38                                         
Covariance Type:            nonrobust                                         
=============================================================================================
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                         8.5556      4.337      1.973      0.049       0.043      17.068
is_changeTeam                -0.5516      0.107     -5.149      0.000      -0.762      -0.341
Age                          -0.6372      0.252     -2.528      0.012      -1.132      -0.143
Age^2                         0.0093      0.004      2.517      0.012       0.002       0.017
AAV_dollar                    0.2097      0.071      2.946      0.003       0.070       0.349
is_1year_contract            -0.0647      0.137     -0.472      0.637      -0.334       0.204
is_FIRST_year_BigContract    -0.0389      0.122     -0.318      0.750      -0.279       0.201
is_END_year                  -1.1478      0.141     -8.132      0.000      -1.425      -0.871
Team_ATL                      0.2533      0.365      0.694      0.488      -0.463       0.969
Team_BAL                     -0.2807      0.471     -0.596      0.551      -1.205       0.643
Team_BOS                      0.4473      0.359      1.244      0.214      -0.258       1.153
Team_CHC                      0.0433      0.336      0.129      0.897      -0.616       0.702
Team_CHW                      0.3520      0.362      0.971      0.332      -0.359       1.063
Team_CIN                     -0.2810      0.367     -0.765      0.444      -1.002       0.440
Team_CLE                      0.4499      0.395      1.140      0.255      -0.325       1.225
Team_COL                     -0.5065      0.383     -1.323      0.186      -1.258       0.245
Team_DET                     -0.2946      0.371     -0.795      0.427      -1.022       0.433
Team_HOU                      0.3058      0.351      0.870      0.385      -0.384       0.996
Team_KC                      -0.0590      0.351     -0.168      0.866      -0.748       0.630
Team_LAA                     -0.3851      0.375     -1.026      0.305      -1.122       0.352
Team_LAD                     -0.1804      0.341     -0.528      0.597      -0.850       0.490
Team_MIA                     -0.1504      0.386     -0.390      0.697      -0.908       0.607
Team_MIL                     -0.0146      0.354     -0.041      0.967      -0.708       0.679
Team_MIN                     -0.2103      0.353     -0.596      0.552      -0.903       0.483
Team_NYM                     -0.4221      0.355     -1.188      0.235      -1.120       0.275
Team_NYY                      0.5286      0.369      1.431      0.153      -0.196       1.253
Team_OAK                      0.6831      0.357      1.915      0.056      -0.017       1.383
Team_PHI                      0.2543      0.355      0.716      0.474      -0.442       0.951
Team_PIT                      0.3039      0.449      0.677      0.499      -0.577       1.185
Team_SD                      -0.1199      0.363     -0.330      0.741      

## 野手

In [23]:
# df_pic = df_pitch.copy(deep=True) # 投手
df_pic = df_bat.copy(deep=True) # 野手
df_pic = df_pic[(df_pic.loc[:,"Season"]<=2021) & (df_pic.loc[:,"Season"]>=2012)]
print(df_pic.Season.unique())


X_cols = df_pic.columns.tolist()
remove_lst = [
 '_idx',
 'Season',
 'Player',
 'Team',
 'WAR',
 'WAR_lag_1_MA3',
 'WAR_SHIRK',
#  'is_changeTeam',
#  'Age',
#  'Age^2',
 'Pos',
 'YRS',
 'YRS_left',
#  'AAV_dollar',
#  'is_1year_contract',
#  'is_FIRST_year_BigContract',
#  'is_END_year',
 'is_left_2years',
 'is_left_3years',
 'is_left_ge4years',
 "type",
 "Player_ID",
     "NewCBA",
#     "is_gt_2020"
]

for col in remove_lst:
    X_cols.remove(col)

# dropna
print(len(df_pic))
df_pic = df_pic[X_cols+["WAR"]].replace([np.inf, -np.inf], np.nan).dropna()
print(len(df_pic))

X = df_pic[X_cols].copy(deep=True)
y = df_pic["WAR"].copy(deep=True)

# OLS
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

[2012 2013 2014 2015 2016 2017 2018 2019 2020 2021]
2496
929


C:\Users\EMCT_Jeff\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    WAR   R-squared:                       0.332
Model:                            OLS   Adj. R-squared:                  0.297
Method:                 Least Squares   F-statistic:                     9.539
Date:                Sat, 11 Sep 2021   Prob (F-statistic):           3.58e-51
Time:                        14:05:02   Log-Likelihood:                -1673.2
No. Observations:                 929   AIC:                             3440.
Df Residuals:                     882   BIC:                             3668.
Df Model:                          46                                         
Covariance Type:            nonrobust                                         
=============================================================================================
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                        12.1428      3.700      3.282      0.001       4.882      19.404
is_changeTeam                -0.6311      0.127     -4.951      0.000      -0.881      -0.381
Age                          -0.9919      0.218     -4.551      0.000      -1.420      -0.564
Age^2                         0.0132      0.003      4.000      0.000       0.007       0.020
AAV_dollar                    0.5226      0.071      7.368      0.000       0.383       0.662
is_1year_contract             0.1864      0.162      1.154      0.249      -0.131       0.503
is_FIRST_year_BigContract    -0.0066      0.133     -0.049      0.961      -0.268       0.255
is_END_year                  -1.2514      0.159     -7.887      0.000      -1.563      -0.940
Team_ATL                     -0.6791      0.346     -1.961      0.050      -1.359       0.001
Team_BAL                     -1.1176      0.422     -2.651      0.008      -1.945      -0.290
Team_BOS                     -0.3679      0.340     -1.081      0.280      -1.036       0.300
Team_CHC                     -0.5277      0.376     -1.403      0.161      -1.266       0.211
Team_CHW                     -0.6269      0.394     -1.591      0.112      -1.400       0.146
Team_CIN                     -0.2593      0.377     -0.687      0.492      -1.000       0.481
Team_CLE                     -0.5463      0.352     -1.552      0.121      -1.237       0.145
Team_COL                     -1.3871      0.394     -3.521      0.000      -2.160      -0.614
Team_DET                     -0.8316      0.393     -2.116      0.035      -1.603      -0.060
Team_HOU                      0.2779      0.389      0.715      0.475      -0.485       1.041
Team_KC                      -0.4574      0.400     -1.143      0.254      -1.243       0.328
Team_LAA                     -0.4033      0.349     -1.154      0.249      -1.089       0.283
Team_LAD                      0.6354      0.390      1.630      0.104      -0.130       1.401
Team_MIA                     -0.8239      0.389     -2.120      0.034      -1.587      -0.061
Team_MIL                      0.0361      0.364      0.099      0.921      -0.678       0.750
Team_MIN                     -0.5931      0.421     -1.407      0.160      -1.420       0.234
Team_NYM                     -0.2632      0.358     -0.735      0.463      -0.966       0.440
Team_NYY                     -0.1852      0.354     -0.523      0.601      -0.879       0.509
Team_OAK                     -0.1222      0.415     -0.294      0.769      -0.937       0.693
Team_PHI                     -0.3289      0.400     -0.821      0.412      -1.115       0.457
Team_PIT                     -0.1561      0.426     -0.366      0.714      -0.993       0.681
Team_SD                      -0.8582      0.414     -2.074      0.038      

### 前5年

In [24]:
# df_pic = df_pitch.copy(deep=True) # 投手
df_pic = df_bat.copy(deep=True) # 野手
df_pic = df_pic[(df_pic.loc[:,"Season"]<=2016) & (df_pic.loc[:,"Season"]>=2012)]
print(df_pic.Season.unique())


X_cols = df_pic.columns.tolist()
remove_lst = [
 '_idx',
 'Season',
 'Player',
 'Team',
 'WAR',
 'WAR_lag_1_MA3',
 'WAR_SHIRK',
#  'is_changeTeam',
#  'Age',
#  'Age^2',
 'Pos',
 'YRS',
 'YRS_left',
#  'AAV_dollar',
#  'is_1year_contract',
#  'is_FIRST_year_BigContract',
#  'is_END_year',
 'is_left_2years',
 'is_left_3years',
 'is_left_ge4years',
 "type",
 "Player_ID",
     "NewCBA",
#     "is_gt_2020"
]

for col in remove_lst:
    X_cols.remove(col)

# dropna
print(len(df_pic))
df_pic = df_pic[X_cols+["WAR"]].replace([np.inf, -np.inf], np.nan).dropna()
print(len(df_pic))

X = df_pic[X_cols].copy(deep=True)
y = df_pic["WAR"].copy(deep=True)

# OLS
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

[2012 2013 2014 2015 2016]
1599
421


C:\Users\EMCT_Jeff\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    WAR   R-squared:                       0.364
Model:                            OLS   Adj. R-squared:                  0.288
Method:                 Least Squares   F-statistic:                     4.775
Date:                Sat, 11 Sep 2021   Prob (F-statistic):           6.48e-18
Time:                        14:05:02   Log-Likelihood:                -763.89
No. Observations:                 421   AIC:                             1620.
Df Residuals:                     375   BIC:                             1806.
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
=============================================================================================
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                         6.7194      5.552      1.210      0.227      -4.197      17.636
is_changeTeam                -0.6695      0.205     -3.260      0.001      -1.073      -0.266
Age                          -0.7967      0.328     -2.428      0.016      -1.442      -0.152
Age^2                         0.0110      0.005      2.217      0.027       0.001       0.021
AAV_dollar                    0.6366      0.111      5.734      0.000       0.418       0.855
is_1year_contract             0.3688      0.265      1.390      0.165      -0.153       0.890
is_FIRST_year_BigContract     0.2384      0.214      1.112      0.267      -0.183       0.660
is_END_year                  -1.3135      0.254     -5.176      0.000      -1.812      -0.815
Team_ATL                     -1.4441      0.591     -2.442      0.015      -2.607      -0.281
Team_BAL                     -0.3256      0.736     -0.442      0.659      -1.774       1.123
Team_BOS                     -0.3820      0.554     -0.690      0.491      -1.471       0.707
Team_CHC                     -0.7264      0.606     -1.199      0.231      -1.918       0.465
Team_CHW                     -0.9115      0.613     -1.487      0.138      -2.117       0.294
Team_CIN                     -0.7326      0.575     -1.273      0.204      -1.864       0.399
Team_CLE                     -0.9667      0.588     -1.645      0.101      -2.122       0.189
Team_COL                     -1.2418      0.656     -1.893      0.059      -2.532       0.048
Team_DET                     -0.6954      0.617     -1.128      0.260      -1.908       0.517
Team_HOU                     -0.1943      0.760     -0.256      0.798      -1.689       1.301
Team_KC                      -1.1032      0.705     -1.566      0.118      -2.489       0.282
Team_LAA                     -0.4569      0.599     -0.763      0.446      -1.635       0.721
Team_LAD                      0.3238      0.660      0.491      0.624      -0.973       1.621
Team_MIA                     -1.1924      0.599     -1.992      0.047      -2.369      -0.015
Team_MIL                     -0.4962      0.591     -0.840      0.402      -1.658       0.666
Team_MIN                     -0.9820      0.683     -1.438      0.151      -2.325       0.361
Team_NYM                      0.2511      0.617      0.407      0.684      -0.963       1.465
Team_NYY                     -0.9985      0.554     -1.803      0.072      -2.087       0.090
Team_OAK                     -0.4603      0.677     -0.680      0.497      -1.792       0.872
Team_PHI                     -0.5842      0.760     -0.769      0.443      -2.078       0.910
Team_PIT                     -0.2054      0.639     -0.321      0.748      -1.463       1.052
Team_SD                      -0.9676      0.693     -1.396      0.164      

### 後5年

In [25]:
# df_pic = df_pitch.copy(deep=True) # 投手
df_pic = df_bat.copy(deep=True) # 野手
df_pic = df_pic[(df_pic.loc[:,"Season"]<=2021) & (df_pic.loc[:,"Season"]>=2017)]
print(df_pic.Season.unique())


X_cols = df_pic.columns.tolist()
remove_lst = [
 '_idx',
 'Season',
 'Player',
 'Team',
 'WAR',
 'WAR_lag_1_MA3',
 'WAR_SHIRK',
#  'is_changeTeam',
#  'Age',
#  'Age^2',
 'Pos',
 'YRS',
 'YRS_left',
#  'AAV_dollar',
#  'is_1year_contract',
#  'is_FIRST_year_BigContract',
#  'is_END_year',
 'is_left_2years',
 'is_left_3years',
 'is_left_ge4years',
 "type",
 "Player_ID",
     "NewCBA",
#     "is_gt_2020"
]

for col in remove_lst:
    X_cols.remove(col)

# dropna
print(len(df_pic))
df_pic = df_pic[X_cols+["WAR"]].replace([np.inf, -np.inf], np.nan).dropna()
print(len(df_pic))

X = df_pic[X_cols].copy(deep=True)
y = df_pic["WAR"].copy(deep=True)

# OLS
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

[2017 2018 2019 2020 2021]
897
508


C:\Users\EMCT_Jeff\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    WAR   R-squared:                       0.388
Model:                            OLS   Adj. R-squared:                  0.326
Method:                 Least Squares   F-statistic:                     6.343
Date:                Sat, 11 Sep 2021   Prob (F-statistic):           2.07e-27
Time:                        14:05:03   Log-Likelihood:                -875.55
No. Observations:                 508   AIC:                             1845.
Df Residuals:                     461   BIC:                             2044.
Df Model:                          46                                         
Covariance Type:            nonrobust                                         
=============================================================================================
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                        19.0415      5.249      3.628      0.000       8.728      29.355
is_changeTeam                -0.5427      0.167     -3.251      0.001      -0.871      -0.215
Age                          -1.2737      0.307     -4.146      0.000      -1.877      -0.670
Age^2                         0.0167      0.005      3.592      0.000       0.008       0.026
AAV_dollar                    0.4247      0.099      4.297      0.000       0.231       0.619
is_1year_contract            -0.0116      0.207     -0.056      0.956      -0.419       0.396
is_FIRST_year_BigContract    -0.3064      0.177     -1.726      0.085      -0.655       0.042
is_END_year                  -1.1155      0.208     -5.354      0.000      -1.525      -0.706
Team_ATL                     -0.1601      0.425     -0.377      0.706      -0.995       0.674
Team_BAL                     -1.5589      0.516     -3.022      0.003      -2.573      -0.545
Team_BOS                     -0.6323      0.434     -1.458      0.146      -1.485       0.220
Team_CHC                     -0.3156      0.489     -0.645      0.519      -1.277       0.645
Team_CHW                     -0.3624      0.542     -0.669      0.504      -1.427       0.702
Team_CIN                      0.1896      0.544      0.348      0.728      -0.880       1.260
Team_CLE                     -0.3184      0.447     -0.712      0.477      -1.198       0.561
Team_COL                     -1.5261      0.492     -3.104      0.002      -2.492      -0.560
Team_DET                     -1.0931      0.530     -2.061      0.040      -2.135      -0.051
Team_HOU                      0.6553      0.452      1.450      0.148      -0.233       1.543
Team_KC                      -0.2172      0.484     -0.448      0.654      -1.169       0.735
Team_LAA                     -0.3992      0.435     -0.918      0.359      -1.254       0.456
Team_LAD                      0.8777      0.484      1.813      0.071      -0.074       1.829
Team_MIA                     -0.6127      0.532     -1.152      0.250      -1.658       0.433
Team_MIL                      0.3424      0.459      0.745      0.457      -0.561       1.245
Team_MIN                     -0.2967      0.548     -0.542      0.588      -1.373       0.780
Team_NYM                     -0.5569      0.440     -1.266      0.206      -1.421       0.307
Team_NYY                      0.5678      0.476      1.194      0.233      -0.367       1.502
Team_OAK                      0.0712      0.539      0.132      0.895      -0.989       1.131
Team_PHI                     -0.1638      0.474     -0.345      0.730      -1.096       0.768
Team_PIT                     -0.2969      0.601     -0.494      0.621      -1.478       0.884
Team_SD                      -0.8948      0.532     -1.682      0.093      

# Model 5 

In [26]:
# SHIRK = β1 NewCBA + One Year Contract + Age + Age2 + Position + ln (Average Salary) + Team + ChangedTeam + Last Year + 2020season + 2021season + ε……… (5)

## 投手

In [27]:
df_pic = df_pitch.copy(deep=True) # 投手
# df_pic = df_bat.copy(deep=True) # 野手
df_pic = df_pic[(df_pic.loc[:,"Season"]<=2021) & (df_pic.loc[:,"Season"]>=2012)]
print(df_pic.Season.unique())


X_cols = df_pic.columns.tolist()
remove_lst = [
 '_idx',
 'Season',
 'Player',
 'Team',
 'WAR',
 'WAR_lag_1_MA3',
 'WAR_SHIRK',
#  'is_changeTeam',
#  'Age',
#  'Age^2',
 'Pos',
 'YRS',
 'YRS_left',
#  'AAV_dollar',
#  'is_1year_contract',
 'is_FIRST_year_BigContract',
#  'is_END_year',
 'is_left_2years',
 'is_left_3years',
 'is_left_ge4years',
 "type",
 "Player_ID",
#      "NewCBA",
#     "is_gt_2020"
]

for col in remove_lst:
    X_cols.remove(col)

# dropna
print(len(df_pic))
df_pic = df_pic[X_cols+["WAR"]].replace([np.inf, -np.inf], np.nan).dropna()
print(len(df_pic))

X = df_pic[X_cols].copy(deep=True)
y = df_pic["WAR"].copy(deep=True)

# OLS
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

[2012 2013 2014 2015 2016 2017 2018 2019 2020 2021]
2487
921


C:\Users\EMCT_Jeff\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    WAR   R-squared:                       0.243
Model:                            OLS   Adj. R-squared:                  0.210
Method:                 Least Squares   F-statistic:                     7.442
Date:                Sat, 11 Sep 2021   Prob (F-statistic):           4.89e-33
Time:                        14:05:03   Log-Likelihood:                -1540.9
No. Observations:                 921   AIC:                             3160.
Df Residuals:                     882   BIC:                             3348.
Df Model:                          38                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 8.5029      4.332      1.963      0.050       0.001      17.005
is_changeTeam        -0.5432      0.104     -5.233      0.000      -0.747      -0.339
Age                  -0.6351      0.252     -2.521      0.012      -1.129      -0.141
Age^2                 0.0093      0.004      2.512      0.012       0.002       0.017
AAV_dollar            0.2078      0.072      2.881      0.004       0.066       0.349
is_1year_contract    -0.0502      0.129     -0.389      0.697      -0.303       0.203
is_END_year          -1.1437      0.140     -8.174      0.000      -1.418      -0.869
Team_ATL              0.2558      0.365      0.702      0.483      -0.460       0.972
Team_BAL             -0.2877      0.471     -0.611      0.541      -1.212       0.636
Team_BOS              0.4542      0.359      1.267      0.206      -0.249       1.158
Team_CHC              0.0422      0.336      0.126      0.900      -0.617       0.701
Team_CHW              0.3574      0.362      0.986      0.324      -0.354       1.068
Team_CIN             -0.2808      0.367     -0.765      0.445      -1.001       0.440
Team_CLE              0.4508      0.395      1.142      0.254      -0.324       1.226
Team_COL             -0.5084      0.383     -1.327      0.185      -1.260       0.244
Team_DET             -0.2870      0.371     -0.774      0.439      -1.015       0.441
Team_HOU              0.3063      0.351      0.871      0.384      -0.384       0.996
Team_KC              -0.0539      0.351     -0.154      0.878      -0.743       0.635
Team_LAA             -0.3798      0.376     -1.011      0.312      -1.117       0.357
Team_LAD             -0.1762      0.341     -0.516      0.606      -0.846       0.494
Team_MIA             -0.1550      0.386     -0.402      0.688      -0.912       0.602
Team_MIL             -0.0146      0.354     -0.041      0.967      -0.709       0.679
Team_MIN             -0.2065      0.353     -0.585      0.559      -0.900       0.487
Team_NYM             -0.4240      0.355     -1.193      0.233      -1.121       0.273
Team_NYY              0.5303      0.369      1.436      0.151      -0.194       1.255
Team_OAK              0.6791      0.357      1.901      0.058      -0.022       1.380
Team_PHI              0.2579      0.355      0.727      0.468      -0.439       0.955
Team_PIT              0.3028      0.449      0.674      0.500      -0.578       1.184
Team_SD              -0.1225      0.363     -0.337      0.736      -0.836       0.591
Team_SEA              0.1867      0.375      0.498      0.618      -0.549       0.922
Team_SF              -0.1077      0.346     -0.311      0.756      -0.787       0.571
Team_STL             -0.4040      0.377     -1.073      0.284      -1.143       0.335
Team_TB              -0.0541      0.417     -0.130      0.897      -0.873

## 野手

In [28]:
# df_pic = df_pitch.copy(deep=True) # 投手
df_pic = df_bat.copy(deep=True) # 野手
df_pic = df_pic[(df_pic.loc[:,"Season"]<=2021) & (df_pic.loc[:,"Season"]>=2012)]
print(df_pic.Season.unique())


X_cols = df_pic.columns.tolist()
remove_lst = [
 '_idx',
 'Season',
 'Player',
 'Team',
 'WAR',
 'WAR_lag_1_MA3',
 'WAR_SHIRK',
#  'is_changeTeam',
#  'Age',
#  'Age^2',
 'Pos',
 'YRS',
 'YRS_left',
#  'AAV_dollar',
#  'is_1year_contract',
 'is_FIRST_year_BigContract',
#  'is_END_year',
 'is_left_2years',
 'is_left_3years',
 'is_left_ge4years',
 "type",
 "Player_ID",
#      "NewCBA",
#     "is_gt_2020"
]

for col in remove_lst:
    X_cols.remove(col)

# dropna
print(len(df_pic))
df_pic = df_pic[X_cols+["WAR"]].replace([np.inf, -np.inf], np.nan).dropna()
print(len(df_pic))

X = df_pic[X_cols].copy(deep=True)
y = df_pic["WAR"].copy(deep=True)

# OLS
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

[2012 2013 2014 2015 2016 2017 2018 2019 2020 2021]
2496
929


C:\Users\EMCT_Jeff\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    WAR   R-squared:                       0.333
Model:                            OLS   Adj. R-squared:                  0.298
Method:                 Least Squares   F-statistic:                     9.577
Date:                Sat, 11 Sep 2021   Prob (F-statistic):           2.12e-51
Time:                        14:05:03   Log-Likelihood:                -1672.6
No. Observations:                 929   AIC:                             3439.
Df Residuals:                     882   BIC:                             3666.
Df Model:                          46                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                11.9633      3.683      3.248      0.001       4.734      19.192
is_changeTeam        -0.6265      0.125     -5.012      0.000      -0.872      -0.381
Age                  -0.9922      0.218     -4.559      0.000      -1.419      -0.565
Age^2                 0.0132      0.003      4.004      0.000       0.007       0.020
AAV_dollar            0.5390      0.072      7.491      0.000       0.398       0.680
is_1year_contract     0.1965      0.153      1.284      0.200      -0.104       0.497
is_END_year          -1.2403      0.157     -7.913      0.000      -1.548      -0.933
Team_ATL             -0.6825      0.346     -1.972      0.049      -1.362      -0.003
Team_BAL             -1.1145      0.421     -2.645      0.008      -1.941      -0.288
Team_BOS             -0.3849      0.340     -1.131      0.258      -1.053       0.283
Team_CHC             -0.5441      0.376     -1.446      0.148      -1.283       0.194
Team_CHW             -0.6626      0.395     -1.678      0.094      -1.437       0.112
Team_CIN             -0.2909      0.378     -0.770      0.442      -1.033       0.451
Team_CLE             -0.5593      0.352     -1.589      0.112      -1.250       0.132
Team_COL             -1.3853      0.394     -3.519      0.000      -2.158      -0.613
Team_DET             -0.8564      0.393     -2.178      0.030      -1.628      -0.085
Team_HOU              0.2802      0.388      0.721      0.471      -0.482       1.043
Team_KC              -0.4629      0.400     -1.157      0.248      -1.248       0.322
Team_LAA             -0.4051      0.349     -1.160      0.246      -1.090       0.280
Team_LAD              0.6341      0.390      1.628      0.104      -0.131       1.399
Team_MIA             -0.8384      0.388     -2.159      0.031      -1.600      -0.076
Team_MIL              0.0135      0.364      0.037      0.970      -0.701       0.728
Team_MIN             -0.6096      0.421     -1.446      0.148      -1.437       0.218
Team_NYM             -0.2693      0.358     -0.752      0.452      -0.972       0.433
Team_NYY             -0.2068      0.354     -0.584      0.559      -0.902       0.488
Team_OAK             -0.1289      0.415     -0.311      0.756      -0.943       0.685
Team_PHI             -0.3366      0.400     -0.841      0.401      -1.122       0.449
Team_PIT             -0.1713      0.426     -0.402      0.688      -1.008       0.665
Team_SD              -0.8803      0.414     -2.126      0.034      -1.693      -0.068
Team_SEA              0.4545      0.403      1.128      0.260      -0.336       1.245
Team_SF              -0.8723      0.382     -2.286      0.022      -1.621      -0.123
Team_STL             -0.2490      0.366     -0.681      0.496      -0.967       0.469
Team_TB              -0.6307      0.417     -1.514      0.130      -1.448

### 前5年

In [29]:
# df_pic = df_pitch.copy(deep=True) # 投手
df_pic = df_bat.copy(deep=True) # 野手
df_pic = df_pic[(df_pic.loc[:,"Season"]<=2016) & (df_pic.loc[:,"Season"]>=2012)]
print(df_pic.Season.unique())


X_cols = df_pic.columns.tolist()
remove_lst = [
 '_idx',
 'Season',
 'Player',
 'Team',
 'WAR',
 'WAR_lag_1_MA3',
 'WAR_SHIRK',
#  'is_changeTeam',
#  'Age',
#  'Age^2',
 'Pos',
 'YRS',
 'YRS_left',
#  'AAV_dollar',
#  'is_1year_contract',
 'is_FIRST_year_BigContract',
#  'is_END_year',
 'is_left_2years',
 'is_left_3years',
 'is_left_ge4years',
 "type",
 "Player_ID",
#      "NewCBA",
#     "is_gt_2020"
]

for col in remove_lst:
    X_cols.remove(col)

# dropna
print(len(df_pic))
df_pic = df_pic[X_cols+["WAR"]].replace([np.inf, -np.inf], np.nan).dropna()
print(len(df_pic))

X = df_pic[X_cols].copy(deep=True)
y = df_pic["WAR"].copy(deep=True)

# OLS
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

[2012 2013 2014 2015 2016]
1599
421


C:\Users\EMCT_Jeff\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    WAR   R-squared:                       0.362
Model:                            OLS   Adj. R-squared:                  0.288
Method:                 Least Squares   F-statistic:                     4.852
Date:                Sat, 11 Sep 2021   Prob (F-statistic):           4.77e-18
Time:                        14:05:03   Log-Likelihood:                -764.58
No. Observations:                 421   AIC:                             1619.
Df Residuals:                     376   BIC:                             1801.
Df Model:                          44                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 7.6141      5.495      1.386      0.167      -3.190      18.418
is_changeTeam        -0.7367      0.196     -3.752      0.000      -1.123      -0.351
Age                  -0.8268      0.327     -2.528      0.012      -1.470      -0.184
Age^2                 0.0114      0.005      2.301      0.022       0.002       0.021
AAV_dollar            0.6230      0.110      5.644      0.000       0.406       0.840
is_1year_contract     0.2590      0.246      1.052      0.294      -0.225       0.743
is_END_year          -1.3759      0.248     -5.558      0.000      -1.863      -0.889
Team_ATL             -1.4249      0.591     -2.410      0.016      -2.588      -0.262
Team_BAL             -0.2846      0.736     -0.387      0.699      -1.731       1.162
Team_BOS             -0.3704      0.554     -0.669      0.504      -1.460       0.719
Team_CHC             -0.7046      0.606     -1.163      0.245      -1.896       0.486
Team_CHW             -0.8803      0.613     -1.437      0.152      -2.085       0.325
Team_CIN             -0.7318      0.576     -1.271      0.204      -1.864       0.400
Team_CLE             -0.9590      0.588     -1.631      0.104      -2.115       0.197
Team_COL             -1.2237      0.656     -1.866      0.063      -2.513       0.066
Team_DET             -0.6689      0.616     -1.085      0.279      -1.881       0.543
Team_HOU             -0.1823      0.760     -0.240      0.811      -1.678       1.313
Team_KC              -1.0761      0.705     -1.527      0.128      -2.461       0.309
Team_LAA             -0.4375      0.599     -0.730      0.466      -1.615       0.740
Team_LAD              0.3516      0.659      0.533      0.594      -0.945       1.648
Team_MIA             -1.1383      0.597     -1.908      0.057      -2.312       0.035
Team_MIL             -0.5090      0.591     -0.861      0.390      -1.671       0.653
Team_MIN             -0.9664      0.683     -1.415      0.158      -2.310       0.377
Team_NYM              0.2541      0.617      0.411      0.681      -0.960       1.468
Team_NYY             -0.9956      0.554     -1.797      0.073      -2.085       0.094
Team_OAK             -0.4589      0.678     -0.677      0.499      -1.791       0.873
Team_PHI             -0.5246      0.758     -0.692      0.489      -2.016       0.966
Team_PIT             -0.1990      0.640     -0.311      0.756      -1.457       1.058
Team_SD              -0.9351      0.693     -1.350      0.178      -2.297       0.427
Team_SEA              0.0221      0.606      0.036      0.971      -1.169       1.214
Team_SF              -1.0973      0.671     -1.636      0.103      -2.416       0.221
Team_STL             -0.1002      0.587     -0.171      0.865      -1.255       1.055
Team_TB              -1.1712      0.646     -1.814      0.071      -2.441

### 後5年

In [30]:
# df_pic = df_pitch.copy(deep=True) # 投手
df_pic = df_bat.copy(deep=True) # 野手
df_pic = df_pic[(df_pic.loc[:,"Season"]<=2021) & (df_pic.loc[:,"Season"]>=2017)]
print(df_pic.Season.unique())


X_cols = df_pic.columns.tolist()
remove_lst = [
 '_idx',
 'Season',
 'Player',
 'Team',
 'WAR',
 'WAR_lag_1_MA3',
 'WAR_SHIRK',
#  'is_changeTeam',
#  'Age',
#  'Age^2',
 'Pos',
 'YRS',
 'YRS_left',
#  'AAV_dollar',
#  'is_1year_contract',
 'is_FIRST_year_BigContract',
#  'is_END_year',
 'is_left_2years',
 'is_left_3years',
 'is_left_ge4years',
 "type",
 "Player_ID",
#      "NewCBA",
#     "is_gt_2020"
]

for col in remove_lst:
    X_cols.remove(col)

# dropna
print(len(df_pic))
df_pic = df_pic[X_cols+["WAR"]].replace([np.inf, -np.inf], np.nan).dropna()
print(len(df_pic))

X = df_pic[X_cols].copy(deep=True)
y = df_pic["WAR"].copy(deep=True)

# OLS
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

[2017 2018 2019 2020 2021]

C:\Users\EMCT_Jeff\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)



897
508


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    WAR   R-squared:                       0.384
Model:                            OLS   Adj. R-squared:                  0.324
Method:                 Least Squares   F-statistic:                     6.390
Date:                Sat, 11 Sep 2021   Prob (F-statistic):           2.87e-27
Time:                        14:05:03   Log-Likelihood:                -877.19
No. Observations:                 508   AIC:                             1846.
Df Residuals:                     462   BIC:                             2041.
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
is_changeTeam        -0.5091      0.166     -3.064      0.002      -0.836      -0.183
Age                  -1.2720      0.308     -4.132      0.000      -1.877      -0.667
Age^2                 0.0168      0.005      3.591      0.000       0.008       0.026
AAV_dollar            0.4394      0.099      4.453      0.000       0.245       0.633
is_1year_contract     0.0843      0.200      0.421      0.674      -0.309       0.478
is_END_year          -1.0738      0.207     -5.178      0.000      -1.481      -0.666
Team_ATL             -0.1278      0.425     -0.301      0.764      -0.963       0.708
Team_BAL             -1.5284      0.517     -2.958      0.003      -2.544      -0.513
Team_BOS             -0.6036      0.434     -1.390      0.165      -1.457       0.250
Team_CHC             -0.2648      0.489     -0.541      0.588      -1.226       0.696
Team_CHW             -0.3835      0.543     -0.706      0.480      -1.450       0.683
Team_CIN              0.2304      0.545      0.423      0.673      -0.841       1.302
Team_CLE             -0.2705      0.447     -0.604      0.546      -1.150       0.609
Team_COL             -1.5073      0.493     -3.060      0.002      -2.475      -0.539
Team_DET             -1.0804      0.531     -2.033      0.043      -2.125      -0.036
Team_HOU              0.6626      0.453      1.463      0.144      -0.227       1.553
Team_KC              -0.2152      0.485     -0.443      0.658      -1.169       0.739
Team_LAA             -0.3682      0.436     -0.845      0.398      -1.224       0.488
Team_LAD              0.8769      0.485      1.807      0.071      -0.077       1.830
Team_MIA             -0.6074      0.533     -1.140      0.255      -1.655       0.440
Team_MIL              0.3566      0.460      0.775      0.439      -0.548       1.261
Team_MIN             -0.3053      0.549     -0.556      0.578      -1.384       0.774
Team_NYM             -0.5789      0.441     -1.314      0.189      -1.445       0.287
Team_NYY              0.5837      0.477      1.225      0.221      -0.353       1.520
Team_OAK              0.1227      0.540      0.227      0.820      -0.938       1.184
Team_PHI             -0.1664      0.475     -0.350      0.726      -1.101       0.768
Team_PIT             -0.3071      0.602     -0.510      0.610      -1.490       0.876
Team_SD              -0.8956      0.533     -1.680      0.094      -1.943       0.152
Team_SEA              0.7698      0.592      1.300      0.194      -0.393       1.933
Team_SF              -0.7178      0.466     -1.540      0.124      -1.634       0.198
Team_STL             -0.2520      0.470     -0.536      0.592      -1.176       0.672
Team_TB              -0.1575      0.566     -0.278      0.781      -1.270       0.955
Team_TEX             -0.0488      0.516     -0.095      0.925      -1.063